In [ ]:
import sys
sys.setrecursionlimit(5000)

In [ ]:
import pandas as pd
df = pd.read_csv('mempool.csv')
data = df.values.tolist()

In [ ]:
#create dependencies between parents and children
import math
i = 0
task_id = {}
indegree = {}
adjacency_matrix = {}
task_fee = {}
task_weight = {}
while i<len(data):
  task_id[data[i][0]] = i
  task_fee[data[i][0]] = data[i][1]
  task_weight[data[i][0]] = data[i][2]
  st = str(data[i][3])
  p = st.split(';')
  if(len(p)==1 and len(st)==3):
      indegree[data[i][0]] = 0
      i = i+1
      continue
  indegree[data[i][0]] = len(p)
  for j in p:
    if j in adjacency_matrix.keys():
        adjacency_matrix[j].append(data[i][0])
    else:
        adjacency_matrix[j] = []
        adjacency_matrix[j].append(data[i][0])
  i = i+1
#del adjacency_matrix['nan']

In [ ]:
global max_fee
global ans 
max_fee = 0
ans = []

In [ ]:
#brute force appproach
#function to generate different combinations of transactions in a block
def helper(task_queue, task, weight_total, current_fee, completed):
    global max_fee
    global ans
    if weight_total > 4000000:
        return
    if(indegree[task] != 0):#all parents not completed
        return
    if(completed[task] == 1):#task already completed
        return
    if(weight_total+task_weight[task]>4000000):#invalid task
        return
    weight_total = weight_total + task_weight[task]#add task to task_queue
    completed[task] = 1
    task_queue.append(task)
    current_fee = current_fee + task_fee[task]
    if(current_fee>max_fee):
        max_fee = current_fee
        ans = task_queue
    if task in adjacency_matrix.keys():
        for j in adjacency_matrix[task]:
            indegree[j] = indegree[j]-1
            helper(task_queue, j, weight_total, current_fee, completed)
            completed[j] = 0
    else:
      for t in indegree:
          if(indegree[t]==0 and completed[t] == 0):
              helper(task_queue, t, weight_total, current_fee, completed)
              completed[t] = 0


In [ ]:
for tsk in indegree:
    if(indegree[tsk] == 0):
        task_queue = []
        current_fee = 0
        weight_total = 0
        completed = {key: 0 for key in indegree}
        helper(task_queue, tsk, weight_total, current_fee, completed)

In [ ]:
print(max_fee)
print(ans)